In [14]:
from fastai import *
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.callbacks.hooks import num_features_model
import cv2
from fastai.callbacks.hooks import num_features_model
from fastai.vision import create_head

In [15]:
from motion.detect_human import BBoxDataset

In [16]:
pretrained = 'models/fastai_bbox_detect_humans_val3.pth'

In [17]:
checkpoint = torch.load(pretrained)

In [18]:
checkpoint.keys()

dict_keys(['model', 'opt'])

In [25]:
weights = checkpoint['model']
[weights[k].shape for k in weights.keys() if 'head' in k]

[torch.Size([4096]),
 torch.Size([4096]),
 torch.Size([4096]),
 torch.Size([4096]),
 torch.Size([]),
 torch.Size([512, 4096]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([]),
 torch.Size([4, 512]),
 torch.Size([4])]

In [20]:
class FlukeDetector2(nn.Module):
    def __init__(self, arch=models.resnet18):
        super().__init__() 
        self.cnn = create_body(arch)
        self.head = create_head(num_features_model(self.cnn) * 2, 4)
        
    def forward(self, im):
        x = self.cnn(im)
        x = self.head(x)
        return x.sigmoid_()

In [21]:
mdl = FlukeDetector2(arch=models.resnet50)

In [33]:
list(mdl.state_dict().keys())[:20]

['cnn.0.weight',
 'cnn.1.weight',
 'cnn.1.bias',
 'cnn.1.running_mean',
 'cnn.1.running_var',
 'cnn.1.num_batches_tracked',
 'cnn.4.0.conv1.weight',
 'cnn.4.0.bn1.weight',
 'cnn.4.0.bn1.bias',
 'cnn.4.0.bn1.running_mean',
 'cnn.4.0.bn1.running_var',
 'cnn.4.0.bn1.num_batches_tracked',
 'cnn.4.0.conv2.weight',
 'cnn.4.0.bn2.weight',
 'cnn.4.0.bn2.bias',
 'cnn.4.0.bn2.running_mean',
 'cnn.4.0.bn2.running_var',
 'cnn.4.0.bn2.num_batches_tracked',
 'cnn.4.1.conv1.weight',
 'cnn.4.1.bn1.weight']

In [30]:
list(weights.keys())[:20][1]

'cnn.1.weight'

In [36]:
mdl.load_state_dict(state_dict=weights, strict=False)

In [37]:
a = torch.rand(4, 3, 500, 500)
mdl(a)

tensor([[0.0602, 0.0236, 0.9241, 0.9811],
        [0.1750, 0.0925, 0.7596, 0.8648],
        [0.2676, 0.2358, 0.7410, 0.7573],
        [0.2904, 0.1856, 0.7748, 0.9065]], grad_fn=<SigmoidBackward>)

In [32]:
a = torch.rand(4, 3, 255, 500)
mdl(a).detach().numpy()

array([[0.655848, 0.382767, 0.509568, 0.584517],
       [0.444238, 0.495169, 0.3545  , 0.322035],
       [0.557973, 0.543674, 0.352894, 0.66705 ],
       [0.506991, 0.464837, 0.556003, 0.51314 ]], dtype=float32)